In [ ]:
"""
Purpose: To check on the status of the soma finder


"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

WARNING - 2020-11-19 06:09:22,738 - trimesh_repair - Need to pip install annotationframeworkclient to repair mesh with pychunkedgraph
WARNING - 2020-11-19 06:09:22,739 - trimesh_io - Need to pip install annotationframeworkclient to use dataset_name parameters


In [3]:
import minfig
du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-19 06:10:15,201 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-19 06:10:15,202 - settings - Setting database.user to celiib
INFO - 2020-11-19 06:10:15,203 - settings - Setting database.password to newceliipass
INFO - 2020-11-19 06:10:15,210 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-19 06:10:15,211 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-19 06:10:15,221 - connection - Connect

Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons
Connecting celiib@at-database.ad.bcm.edu:3306


# Chccking the current progress of Soma Extraction

In [18]:
dj.config["display.limit"] = 50

INFO - 2020-11-19 06:13:31,994 - settings - Setting display.limit to 50


In [31]:
minnie.BaylorSegmentCentroid() & "n_vertices>0"

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),soma_index index given to this soma to account for multiple somas in one base semgnet,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),n_vertices number of vertices,n_faces number of faces,soma_vertices array of vertices,soma_faces array of faces,multiplicity the number of somas found for this base segment,sdf sdf width value for the soma,max_side_ratio the maximum of the side length ratios used for check if soma,bbox_volume_ratio ratio of bbox (axis aligned) volume to mesh volume to use for check if soma,run_time the amount of time to run (seconds)
864691135544355880,0,0.25,1,166938,159770,15533,1681,3266,=BLOB=,=BLOB=,1,0.848,1.055,2.515,44.9078


In [33]:
(schema.jobs & "table_name='__baylor_segment_centroid'")#.delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp


In [26]:
curr_seg_id = 864691134410249536
new_mesh = du.fetch_segment_id_mesh(curr_seg_id)

INFO - 2020-11-19 06:14:49,441 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-19 06:14:49,681 - settings - Setting enable_python_native_blobs to True


In [27]:
import neuron_visualizations as nviz
nviz.plot_objects(new_mesh)

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))
